# Lab 1 - First steps with Spark

First:

- open a new tab
- open a terminal 
- Execute `watch kubectl get pods` to see Pods running in real time during the whole tutorial

At this end of this lab :

- export your notebook : File > Export Notebook As ... > Export Notebook To HTML
- And turn your Jupyter service off !

Let's simply explain the Spark context. What's happening when I use Spark ? What is Spark ? What can I do with it ? Why is it dope ? Let's get going !

## Understand some basic principles of Spark context

### What is Spark ?


Let's keep it simple. 

The intuitive way: Spark distributes tasks to different Worker nodes (machine executing the tasks).  
The Kubernetes way: Spark distributes tasks to different containers. The location of containers among the different worker nodes is then handled by Kubernetes. The more computation power you need, the more containers get created. It's a smooth way to save Worker nodes resources.
With Kubernetes, we just speak in terms of Pods rathen than containers.

Pretty handy right ? If you want to have a deeper look on how it's done, you can by clicking on <font size="3" color="darkgreen"><b>If you want more </b></font> but you still have some work to do so don't waste your time ! You can keep it to the end

<details>    
<summary>
    <font size="3" color="darkgreen"><b>If you want more ! </b></font>
</summary>

### Spark Context

The **Spark Context** is an object that hides the complexity of the underlying infrastructure to perform computation from the Data Scientist.

This Spark context is a JVM process that gives access to a Spark **driver** which schedules the tasks and spans tasks across Worker nodes through Executors.
In brief, the Spark driver communicates with all the Worker nodes. 

Each Worker node consists of one or more Executor(s) who are responsible for running the Task. Executors register themselves with Driver. The Driver has all the information about the Executors at all the time.

This working combination of Driver and Workers is known as **Spark Application**.

JVM: Java virtual machine that load, verifies and executes Java bytecode.

![](../First-steps-with-cloud-computing/img/spark_context.png)

The Spark Application is launched with the help of the **Cluster Manager**. Spark is dependent on the **Cluster Manager** to launch the Executors and also the Driver (in Cluster mode). 

Spark can be run with any of the 5 following **Cluster Manager** :

    - local : Driver use CPU threads on your local machine
    - Spark Standalone Mode : A basic resource manager provided by Spark
    - YARN : the historical resource manager commonly used in traditional Big Data infrastructures (Courdera or Hortonworks cluster for instance)
    - Mesos : the resource manager from Berkeley University
    - Kubernetes : the game changing containers orchestrator

## Spark on a Kubernetes cluster

![](../First-steps-with-cloud-computing/img/k8spark-cluster-mode.png)

The Spark driver runs on a Kubernetes Pod and creates executors running within Kubernetes pods. In simple terms, Spark driver is in a Pod and it distributes tasks to different Pods. The location of Pods among the different worker nodes is then handled by Kubernetes. 

The Kubernetes **Scheduler** checks if pods are assigned to nodes. For every Pod that the scheduler discovers, the scheduler is responsible of finding the best node for that Pod to run on.  
The Kubernetes **API Server** allows to interact with Pods. It schedules executor Pods by creating or deleting Pods.


</detail>



## Creating a Spark session

Write in a new cell :
- execute the 2 following cells to initialize a Spark session
- `spark`  to check if everything is ok

In [ ]:
import os

In [ ]:
from pyspark.sql import SparkSession

spark = (SparkSession 
         .builder
         # default url of the internally accessed Kubernetes API
         # (This Jupyter notebook service is itself a Kubernetes Pod)
         .master("k8s://https://kubernetes.default.svc:443")
         # Executors spark docker image: for simplicity reasons, this jupyter notebook is reused 
         .config("spark.kubernetes.container.image", os.environ['IMAGE_NAME'])
         # Name of the Kubernetes namespace
         .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])
         # Allocated memory to the JVM
         # Stay careful, by default, the Kubernetes pods has a higher limit which depends on other parameters.
         .config("spark.executor.memory", "4g")
         .config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])
         # dynamic allocation configuration
         .config("spark.dynamicAllocation.enabled","true")
         .config("spark.dynamicAllocation.initialExecutors","1")
         .config("spark.dynamicAllocation.minExecutors","1")
         .config("spark.dynamicAllocation.maxExecutors","60")
         # Ratio match the number of pods to create for a given number of parallel tasks 
         # (100 parallel, ratio of 1, one aims at 100 pods, with 0.5 it would be 50 pods)
         .config("spark.dynamicAllocation.executorAllocationRatio","1")
         .config("spark.dynamicAllocation.shuffleTracking.enabled","true")
         .getOrCreate()
        )


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Non-dynamic allocation: if you want to choose manually the number of pods (not recommended) </b></font>
</summary>
    
from pyspark.sql import SparkSession

spark = (SparkSession 
         .builder
         # default url of the internally accessed Kubernetes API
         # (This Jupyter notebook service is itself a Kubernetes Pod)
         .master("k8s://https://kubernetes.default.svc:443")
         # Executors spark docker image: for simplicity reasons, this jupyter notebook is reused 
         .config("spark.kubernetes.container.image", os.environ['IMAGE_NAME'])
         # Name of the Kubernetes namespace
         .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])
         # Number of Spark executors, there will be launched as much Kubernetes pods as the entered integer.
         .config("spark.executor.instances", "5")
         # Allocated memory to the JVM
         # Stay careful, by default, the Kubernetes pods has a higher limit which depends on other parameters.
         .config("spark.executor.memory", "4g")
         .config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])a
         .getOrCreate()
        )




In [5]:
#Spark session
spark

Now you can check out your terminal and see that a pyspark Pod has been created

## 💾First steps with Spark - Data importation

Spark's main object class is the DataFrame, which is a distributed table. It is analogous to R's or Python (Pandas)'s data frames: one row represents an observation, one column represents a variable. But contrary to R or Python, Spark's DataFrames can be distributed over hundred of nodes.

Spark support multiple data formats, and multiple  ways to load them.

- data format : csv, json, parquet (an open source column oriented format)
- can read archive files
- schema detection or user defined schema. For static data, like a json file, schema detection can be use with good results.

Spark has multiple syntaxes to import data. Some are simple with no customisation, others are more complexes but you can specify options.

The simplest syntaxes to load a json or a csv file are :

```python
# JSON
json_df = spark.read.json([location of the file])
# csv
csv_df = spark.read.csv([location of the file])

```

In the future, you may consult the [Data Source documentation](https://spark.apache.org/docs/latest/sql-data-sources.html) to have the complete description of Spark's reading abilities.

The data you will use in this lab are real data from the twitter [sampled stream API](https://developer.twitter.com/en/docs/twitter-api/tweets/sampled-stream/introduction) and [filtered stream API](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/introduction). The tweets folder contains more than 50 files and more than 2 million tweets. The tweets was collected between the 14/04/2021 and the 18/04/2021. The total collection time was less than 10 hours.

---

### ✍Hands-on 1  - Data importation

- Load the json file store here : `s3a://nrandriamanana/diffusion/formation/data/tweets/tweets20220414-090219.jsonl.gz` and name you data frame `df_tweet`

  <small> ⚙️ This file is an a `JSONL` (JSON-line) format, which means that each line of it is a JSON object. A JSON object is just a Python dictionary or a JavaScript object and looks like this: `{ key1: value1, key2: ["array", "of", "many values]}`). This file has been compressed into a `GZ` archive, hence the `.jsonl.gz` ending. Also this file is not magically appearing in your S3 storage. It is hosted on one of your teacher's bucket and has been made public, so that you can access it.</small>

- It's possible to load multiple file in a unique DataFrame. It's useful when you have daily files and want to process them all. It's the same syntax as the previous one, just specify a folder. Like `s3a://nrandriamanana/diffusion/formation/data/tweets/`. Name you DataFrame `df_tweet_big`

Just after you manage to load your file, check out your terminal again and you'll see Pod popping. The more your code requires resources, the more Spark will create Pods

In [20]:
# DataFrame creation
df_tweet = 
df_tweet_big = 

# caching
df_tweet.cache()
df_tweet_big.cache()

2022-07-13 13:43:11,566 WARN execution.CacheManager: Asked to cache already cached data.
2022-07-13 13:43:11,566 WARN execution.CacheManager: Asked to cache already cached data.


DataFrame[data: struct<author_id:string,created_at:string,entities:struct<annotations:array<struct<end:bigint,normalized_text:string,probability:double,start:bigint,type:string>>,cashtags:array<struct<end:bigint,start:bigint,tag:string>>,hashtags:array<struct<end:bigint,start:bigint,tag:string>>,mentions:array<struct<end:bigint,id:string,start:bigint,username:string>>,urls:array<struct<description:string,display_url:string,end:bigint,expanded_url:string,images:array<struct<height:bigint,url:string,width:bigint>>,start:bigint,status:bigint,title:string,unwound_url:string,url:string>>>,id:string,lang:string,possibly_sensitive:boolean,public_metrics:struct<like_count:bigint,quote_count:bigint,reply_count:bigint,retweet_count:bigint>,source:string,text:string,withheld:struct<copyright:boolean,country_codes:array<string>>>, includes: struct<users:array<struct<created_at:string,id:string,name:string,username:string,verified:boolean,withheld:struct<country_codes:array<string>>>>>]

In [ ]:
#Solution
%load -r 1-7 solutions.py

## 🥉Data frame basic manipulations

If DataFrames are immutable, they can however be **_transformed_** in other DataFrames, in the sense that a modified copy is returned. Such **transformations** include: filtering, sampling, dropping columns, selecting columns, adding new columns...

First, you can get information about the columns with:

```python
df.columns       # get the column names
df.schema        # get the column names and their respective type
df.printSchema() # same, but human-readable
```

You can select columns with the `select()` method. It takes as argument a list of column name. For example :

```python
df_with_less_columns = df\
  .select("variable3","variable_four","variable-6")

# Yes, you do need the ugly \ at the end of the line,
# if you want to chain methods between lines in Python
```

You can get nested columns easily with :

```python
df.select("parentField.nestedField")
```

To filter data you could use the `filter()` method. It take as input an expression that gets evaluated for each observation and should return a boolean. Sampling is performed with the `sample()` method. For example :

```python
df_with_less_rows = df\
  .sample(fraction=0.001)\
  .filter(df.variable1=="value")\
  .show(10)
```

As said before your data are distributed over multiple nodes (executors) and data inside a node are split into partitions. Then each transformations will be run in parallel. They are called *narrow transformation* For example, to sample a DataFrame, Spark sample every partitions in parallel because sample all partition produce the sample DataFrame. For some transformations, like `groupBy()` it's impossible, and it's cannot be run in parallel.

![](https://raw.githubusercontent.com/HealerMikado/panorama_big_data_2021/main/labs/lab%202%20-%20first%20steps%20with%20Spark/img/spark_exemple1_pipeline.png)

<!-- take() collect() limit() first() show() -->
<!-- lien vers la doc https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#dataframe-apis -->

### 😴Lazy evaluation

This is because Spark has what is known as **lazy evaluation**, in the sense that it will wait as much as it can before performing the actual computation. Said otherwise, when you run an instruction such as:

```python
tweet_author_hashtags = df_tweet_big.select("auteur","hashtags")
```

... you are not executing anything! Rather, you are building an **execution plan**, to be realised later.

Spark is quite extreme in its laziness, since only a handful of methods called **actions**, by opposition to **transformations**, will trigger an execution. The most notable are:

1. `collect()`, explicitly asking Spark to fetch the resulting rows instead of to lazily wait for more instructions,
2. `take(n)`, asking for `n` first rows
3. `first()`, an alias for `take(1)`
4. `show()` and `show(n)`, human-friendly alternatives[^5]
5. `count()`, asking for the numbers of rows
6. all the "write" methods (write on file, write to database), see [here](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#input-and-output) for the list

[^5]: `first()` is exactly `take(1)` ([ref]( https://stackoverflow.com/questions/37495039/difference-between-spark-rdds-take1-and-first)) and show prints the result instead of returning it as a list of rows ([ref](https://stackoverflow.com/questions/53884994/what-is-the-difference-between-dataframe-show-and-dataframe-take-in-spark-t))

**This has advantages:** on huge data, you don't want to accidently perform a computation that is not needed. Also, Spark can optimize each **stage** of the execution in regard to what comes next. For instance, filters will be executed as early as possible, since it diminishes the number of rows on which to perform later operations. On the contrary, joins are very computation-intense and will be executed as late as possible. The resulting **execution plan** consists in a **directed acyclic graph** (DAG) that contains the tree of all required actions for a specific computation, ordered in the most effective fashion.

**This has also drawbacks.** Since the computation is optimized for the end result, the intermediate stages are discarded by default. So if you need a DataFrame multiple times, you have to cache it in memory because if you don't Spark will recompute it every single time. 

---

### ✍Hands-on 2 - Data frame basic manipulations

- How many rows have your two DataFrame ?

Printing schema and some rows

In [7]:
df_tweet.[some action](5)
df_tweet.=[some action]()
df_tweet_big.[some action](5)
df_tweet_big.[some action]()


+--------------------+--------------------+
|                data|            includes|
+--------------------+--------------------+
|{1037039682822070...|{[{2018-09-04T18:...|
|{1391192446613160...|{[{2021-05-09T00:...|
|{3498010513, 2022...|{[{2015-09-08T23:...|
|{1203504398321647...|{[{2019-12-08T02:...|
|{3111644864, 2022...|{[{2015-03-27T20:...|
+--------------------+--------------------+
only showing top 5 rows

root
 |-- data: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- entities: struct (nullable = true)
 |    |    |-- annotations: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end: long (nullable = true)
 |    |    |    |    |-- normalized_text: string (nullable = true)
 |    |    |    |    |-- probability: double (nullable = true)
 |    |    |    |    |-- start: long (nullable = true)
 |    |    |    |    |-- type: string (nullable = tr

+--------------------+--------------------+
|                data|            includes|
+--------------------+--------------------+
|{8566371157357076...|{[{2017-04-24T22:...|
|{1100882149060812...|{[{2019-02-27T22:...|
|{1647610112, 2022...|{[{2013-08-05T11:...|
|{1227057358683066...|{[{2020-02-11T02:...|
|{287239880, 2022-...|{[{2011-04-24T16:...|
+--------------------+--------------------+
only showing top 5 rows

root
 |-- data: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- entities: struct (nullable = true)
 |    |    |-- annotations: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end: long (nullable = true)
 |    |    |    |    |-- normalized_text: string (nullable = true)
 |    |    |    |    |-- probability: double (nullable = true)
 |    |    |    |    |-- start: long (nullable = true)
 |    |    |    |    |-- type: string (nullable = tr

In [ ]:
%load -r 9-12 solutions.py

In [9]:
print(f'the small DF has {df_tweet.[some action]()} rows')
print(f'the big DF has {df_tweet_big.[some action]()} rows')

the small DF has 10000 rows
the big DF has 710000 rows


In [ ]:
%load -r 14-15 solutions.py

- Sample `df_tweet_big` and keep only 10% of it. Create a new DataFrame named `df_tweet_sampled`. If computations take too long on the full DataFrame, use this one instead or add a sample transformation in your expression. 

In [ ]:
df_tweet_sampled = 

In [ ]:
%load -r 17-17 solutions.py

- Define a DataFrame `tweet_author_hashtags`  with only the `auteur` and `hashtags` columns

In [ ]:
tweet_author_hashtags = 
tweet_author_hashtags.[some action](5)

In [ ]:
%load -r 19-20 solutions.py

- Print (few lines of) a DataFrame with only the `auteur`, `mentions`, and `urls` columns. (`mentions` and `urls` are both nested columns in `entities`.)

In [ ]:
%load -r 22-22 solutions.py

- Filter your first DataFrame and keep only tweets with more than 1 like. Give a name for this new, transformed DataFrame and print. Print (few lines of) it.

In [ ]:
%load -r 24-24 solutions.pyb

## 🥈Basic DataFrame column manipulation 

You can add/update/rename column of a DataFrame with spark :

- Drop : `df.drop(columnName : str )`
- Rename : `df.withColumnRenamed(oldName : str, newName : str)`
- Add/update : `df.withColumn(columnName : str, columnExpression)` 

For example

```python
tweet_df_with_like_rt_ratio = tweet_df\
  .withColumn(        # computes new variable
    "like_rt_ratio", # like_rt_ratio "OVERCONFIDENCE"
    (tweet_df.like_count /flights.retweet_count
   )

```

See [here](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#functions) for the list of all functions available in an expression.

### ✍Hands-on 3 - Basic DataFrame column manipulation  

- Define a DataFrame with a column names `interaction_count`. This column is the sum of `like_count`, `reply_count` and `retweet_count`.

In [ ]:
%load -r 26-27 solutions.py

- Update the DataFrame you imported at the beginning of this lab and drop the `other` column

In [ ]:
%load -r 29-29 solutions.py

## 🥇Advance DataFrame column manipulation 

### 🥽Array manipulation

Some columns often contain arrays (lists) of values instead of just one value. This may seem surprising but this actually quite natural. For instance, you may create an array of words from a text, or generate a list of random numbers for each observation, etc.

You may **create array of values** with:

- `split(text : string, delimiter : string)`, turning a text into an array of strings

You may **use array of values** with:

- `size(array : Array)`, getting the number of elements

- `array_contains(inputArray : Array, value : any)`, checking if some value appears

- `explode(array : Array)`, unnesting an array and duplicating other values. For instance it if use `explode()` over the hashtags value of this DataFrame:

  | Auteur | Contenu                             | Hashtags         |
  | ------ | ----------------------------------- | ---------------- |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] |
  | Alice  | Just finished #MHrise, best MH ever | [MHrise]         |

  I will get :

  | Auteur | Contenu                             | Hashtags         | Hashtag |
  | ------ | ----------------------------------- | ---------------- | ------- |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] | Spark   |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] | bigdata |
  | Alice  | Just finished #MHrise, best MH ever | [MHrise]         | MHrise  |

  

All this function must be imported first :

```python
from pyspark.sql.functions import split, explode, size, array_contains
```

Do not forget, to create a new column, you should use `withColumn()`. For example : 

```python
df.withColumn("new column", explode("array"))
```

#### ✍Hands-on 4 - Array manipulation 

- Keep all the tweets with hashtags and for each remaining line, split the hashtag text into an array of hashtags

In [ ]:
from pyspark.sql.functions import split, explode, size, array_contains

In [ ]:
%load -r 31-31 solutions.py

- Create a new column with the number of words of the `contenu` column. (Use `split()` + `size()`)

In [ ]:
%load -r 33-33 solutions.py

- Count how many tweet contain the `covid19` hashtag (use the `count()` action)

In [ ]:
%load -r 35-35 solutions.py

### 🥼User defined function

For more very specific column manipulation you will need Spark's `udf()` function (*User Defined Function*). It can be useful if you Spark does not provide a feature you want. But Spark is a popular and active project, so before coding an udf, go check the documentation. For instance for natural language processing, Spark already has some [functions](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.feature.Tokenizer.html#pyspark.ml.feature.Tokenizer). Last things, python udf can lead to performance issues (see https://stackoverflow.com/a/38297050) and learning a little bit of scala or java can be a good idea.

For example :

```python
# !!!! DOES NOT WORK !!!!
def to_lower_case(string):
	return string.lower()
	
df.withColumn("tweet_lower_case", to_lower_case(df.contenu))
```

will just crash. Keep in mind that Spark is a distributed system, and that Python is only installed on the central node, as a convenience to let you execute instructions on the executor nodes. But by default, pure Python functions can only be executed where Python is installed! We need `udf()` to enable Spark to send Python instructions to the worker nodes.

Let us see how it is done :

```python
# imports
from pyspark.sql.functions import udf
from pyspark.sql.functions import explode
from pyspark.sql.types import StringType

# pure python functions
def to_lower_case(string):
    return string.lower()

# user definid function
to_lower_case_udf = udf(
    lambda x: to_lower_case(x), StringType()
) #we use a lambda function to create the udf.

# df manipulation
df_tweet_small\
  .select("auteur","hashtags")\
  .filter("size(hashtags)!=0")\
  .withColumn("hashtag", explode("hashtags"))\
  .withColumn("hashtag", to_lower_case_udf("hashtag")).show(10)
```

---

#### ✍Hands-on 5 - User defined function 

- Create an user defined function that counts how many words a tweet contains. (your function will return an `IntegerType` and not a `StringType`)

In [ ]:
# imports
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# pure python functions
def word_count(string):
    return ...

# user definid function
word_count_udf = udf(
    lambda x: word_count(x), IntegerType()
) #we use a lambda function to create the udf.

# df manipulation
df_tweet_big\
  .withColumn("word_count", word_count_udf("contenu")).show(10)

In [ ]:
%load -r 37-48 solutions.py

## 🔩Aggregation functions

Spark offer a variety of aggregation functions :

- `count(column : string)` will count every not null value of the specify column. You cant use `count(1)` of `count("*")` to count every line (even row with only null values)

- `countDisctinct(column : string)` and `approx_count_distinct(column : string, percent_error: float)`. If the exact number is irrelevant, `approx_count_distinct()`should be preferred.

  Counting distinct elements cannot be done in parallel, and need a lot data transfer. But if you only need an approximation, there is a algorithm, named hyper-log-log (more info [here](https://databricks.com/fr/blog/2016/05/19/approximate-algorithms-in-apache-spark-hyperloglog-and-quantiles.html)) that can be parallelized. 

  ```python
  from pyspark.sql.functions import count, countDistinct, approx_count_distinct
  
  df.select(count("col1")).show()
  df.select(countDistinct("col1")).show()
  df.select(approx_count_distinct("col1"), 0.1).show()
  ```

- You have access to all other common functions `min()`, `max()`, `first()`, `last()`, `sum()`, `sumDistinct()`, `avg()` etc (you should import them first `from pyspark.sql.functions import min, max, avg, first, last, sum, sumDistinct`) 

---

### ✍Hands-on 6 - Aggregation functions

- What are the min, max, average of `interaction_count`

In [ ]:
from pyspark.sql.functions import min, max, avg, first, last, sum, sumDistinct

In [ ]:
%load -r 50-50 solutions.py

- How many tweets have hashtags ? Distinct hashtags ? Try the approximative count with 0.1 and 0.01as maximum estimation error allowed.

In [ ]:
from pyspark.sql.functions import count, countDistinct, approx_count_distinct

In [ ]:
%load -r 52-52 solutions.py

## Grouping functions

Like SQL you can group row by a criteria with Spark. Just use the `groupBy(column : string)` method. Then you can compute some aggregation over those groups.

```python
df.groupBy("col1").agg(
  count("col2").alias("quantity") # alias is use to specify the name of the new column
).show() 
```

The `agg()` method can take multiples argument to compute multiple aggregation at once.

```python
df.groupBy("col1").agg(
	count("col2").alias("quantity"), min("col2").alias("min"), avg("col3").alias("avg3") ).show()
```

Aggregation and grouping transformations work differently than the previous method like `filter()`, `select()`, `withColumn()` etc. Those transformations cannot be run over each partitions in parallel, and need to transfer data between partitions and executors.  They are called "wide transformations"

<img src="https://raw.githubusercontent.com/HealerMikado/panorama_big_data_2021/main/labs/lab%202%20-%20first%20steps%20with%20Spark/img/spark_exemple2_pipeline.png" style="zoom:30%;" />
---

### ✍Hands-on 7 - Grouping functions

- Compute a daframe with the min, max and average retweet of each `auteur`. Then order it by the max number of retweet in descending order by . To do that you can use the following syntax

  ```python
  from pyspark.sql.functions import desc
  df.orderBy(desc("col"))
  ```

In [ ]:
%load -r 54-55 solutions.py

## 🔌Spark SQL

Spark understand SQL statement. It's not a hack nor a workaround to use SQL in Spark, it's one a the more powerful feature in Spark. To use SQL in you need :

1. Register a view pointing to your DataFrame

   ```python
   my_df.createOrReplaceTempView(viewName : str)
   ```

2. Use the sql function

   ```python
   spark.sql("""
   You sql statment
   """)
   ```

   You could manipulate every registered DataFrame by their view name with plain SQL.

In fact you can do most of this tutorial without any knowledge in PySpark nor Spark. Lot of things can be done in Sparkk only by only knowing SQL and how to use it in Spark. 

### ✍Hands-on 8 - Spark SQL

- How many tweets have hashtags ? Distinct hashtags ? 

In [ ]:
%load -r 57-63 solutions.py

- Compute a dataframe with the min, max and average retweet of each `auteur` using Spark SQL

In [ ]:
%load -r 65-71 solutions.py

## Joins in Spark

Like a SQL, Spark can join two dataset  by comparing the value of one or more keys using joins. Joins are by nature wide transformation, so data will be transferred between executors and will take time. But Spark will distinct two cases :

- Big table to big table join
- Big table to small table join
- (The case small table to small table is irrelevant for big data)

And optimize the join according to the actual case. (For more information Spark: The Definitive Guide pages 148 - 151)

Doing a join is pretty easy. You need :

- At least two DataFrames (obviously) with columns with the same keys
- a join expression
- the join transformation

For instance :

```python
# Creation of 3 small DF
person=spark.createDataFrame([
    (0,"Bill Chambers",0,[100])
    ,(1,"Matei Zaharia",1,[500,250,100])
    ,(2,"Michael Armbrust",1,[250,100])])\
	.toDF("id","name","graduate_program","spark_status")
graduateProgram=spark.createDataFrame([
    (0,"Masters","School of Information","UC Berkeley")
    ,(2,"Masters","EECS","UC Berkeley"),(1,"Ph.D.","EECS","UC Berkeley")])\
	.toDF("id","degree","department","school")
sparkStatus=spark.createDataFrame([
    (500,"Vice President")
    ,(250,"PMC Member")
    ,(100,"Contributor")])\
	.toDF("id","status")
    
# A join expression
joinExpression=person["graduate_program"]==graduateProgram['id']

# The join transformation in action
person.join(graduateProgram, joinExpression).show()
```

By default Spark compute inner joins, but you can pass a third argument to the join transformation with its type. You can do :

- Inner joins : by default or with the "inner" argument
- Outer joins : "outer" argument
- Left / right outer joins : "left_outer", "right_outer" argument
- Left semi join : it's more a filter than a join. It only keep the row in the left DataFrame that have a match in the right DataFrame. Use he "left_semi" argument
- Left anti join : The opposite of the previous one. Use he "left_anti" argument
- You can do cross joins to, but it's a very bad idea to do so, so please don't !

### ✍ Hands-on 9 - Joins in Spark

- Import the files stored in `s3n://spark-lab-input-data-ensai20202021/users/` in a DataFrame  and its informations to your DataFrame. Filter your new DataFrame to only keep verified user (`verified == True`) and group by user and get the most active user of your DataFrame.

In [25]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, TimestampType, BooleanType
schema = StructType([ \
    StructField("created_at",TimestampType(),True), \
    StructField("id",StringType(),True), \
    StructField("name",StringType(),True), \
    StructField("username", StringType(), True), \
    StructField("withheld",StructType([ 
        StructField("country_codes", ArrayType(StringType(),True)),\
        StructField("scope", StringType(), True)  \
    ])),
    StructField("verified",BooleanType(),True)             
  ])

In [26]:
df_user_big = 
df_user_big.cache()
df_user_big.[some action](5)

+--------------------+-------------------+---------------------+--------------+--------+--------+
|          created_at|                 id|                 name|      username|verified|withheld|
+--------------------+-------------------+---------------------+--------------+--------+--------+
|2017-06-12T04:24:...| 874120224055803904|            fengx2017|     fengx2017|   false|    null|
|2012-05-24T07:43:...|          589010172|   BLOODSUCKER&MISERY|     BSRMISERY|   false|    null|
|2017-09-04T12:24:...| 904681576016969738|いわよだ みとくん！🍋|yodayoda_renka|   false|    null|
|2019-03-25T12:29:...|1110156812714565637|     a bug life🦋🐛🌼|      lumyai04|   false|    null|
|2021-01-20T15:57:...|1351921566070652930| ◞ ⁽ ᵐⁱᵇᵘᵘᵗʸ ⁾ #Lo...|   mibukaboomm|   false|    null|
+--------------------+-------------------+---------------------+--------------+--------+--------+
only showing top 5 rows



2022-07-13 13:50:27,300 WARN execution.CacheManager: Asked to cache already cached data.


In [ ]:
%load -r 85-87 solutions.py

In [33]:
df_user_big.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- withheld: struct (nullable = true)
 |    |-- country_codes: array (nullable = true)
 |    |    |-- element: string (containsNull = true)



In [ ]:
%load -r 89-89 solutions.py

In [48]:
# count tweets created after 2019-01-01
from pyspark.sql.functions import to_date, lit


0

In [2]:
%load -r 91-91 solutions.py

In [29]:
#keep verified user


5267

In [ ]:
%load -r 93-93 solutions.py

In [30]:
# join user and tweets dataframes

+----------+---+----+--------+--------+--------+----+--------+
|created_at| id|name|username|verified|withheld|data|includes|
+----------+---+----+--------+--------+--------+----+--------+
+----------+---+----+--------+--------+--------+----+--------+



In [3]:
%load -r 95-96 solutions.py

In [31]:
#count number of verified tweets in the joined table

0

In [4]:
%load -r 98-98 solutions.py


**DO NOT FORGET TO TURN YOUR SERVICE OFF!**

**DO NOT FORGET TO TURN YOUR SERVICE OFF!**

**DO NOT FORGET TO TURN YOUR SERVICE OFF!**

**DO NOT FORGET TO TURN YOUR SERVICE OFF!**

**DO NOT FORGET TO TURN YOUR SERVICE OFF!**

**DO NOT FORGET TO TURN YOUR SERVICE OFF!**

**DO NOT FORGET TO TURN YOUR SERVICE OFF!**

**DO NOT FORGET TO TURN YOUR SERVICE OFF!**
